## Creation of the variables used in the classification

In [1]:
import pandas as pd

In [2]:
data_pan15_train = pd.read_csv(r'C:\Users\Sten\Documents\EUR BIM\thesis\data\data\raw_data\PAN_15_training.csv')

In [3]:
data_pan15_test = pd.read_csv(r'C:\Users\Sten\Documents\EUR BIM\thesis\data\data\raw_data\PAN_15_test.csv')

In [2]:
data_pan17_train = pd.read_csv(r'C:\Users\Sten\Documents\EUR BIM\thesis\data\data\raw_data\PAN_17_training.csv')

In [3]:
data_pan17_test = pd.read_csv(r'C:\Users\Sten\Documents\EUR BIM\thesis\data\data\raw_data\PAN_17_test.csv')

In [6]:
combined_pan17_train = data_pan17_train.groupby('author').agg({
    'text': ' '.join,
    'gender': 'first'
}).reset_index()
combined_pan17_test = data_pan17_test.groupby('author').agg({
    'text': ' '.join,
    'gender': 'first'
}).reset_index()

NameError: name 'data_pan17_train' is not defined

In [7]:
combined_pan15_train = data_pan15_train.groupby('author').agg({
    'text': ' '.join,
    'gender': 'first',
    'age': 'first'
}).reset_index()
combined_pan15_test = data_pan15_test.groupby('author').agg({
    'text': ' '.join,
    'gender': 'first',
    'age': 'first'
}).reset_index()

In [8]:
combined_pan15_train

,author,text,gender,age
0,02ae95de-7ee3-453a-978d-25d28b3f1a88,Things I want for my business cards but are to...,M,25-34
1,03f72f70-7a00-4dbb-93b7-6c7f65954fc5,"""If you fight for a cause then fight for a cau...",M,25-34
2,0aa7bace-924c-40fb-a2e5-3c7012ede244,@username that tweet was mainly for you ;-) ch...,M,35-49
3,0b66092a-440e-4755-a624-759a580a1c70,@username 'red' and 'blue' and then you wait t...,F,25-34
4,1094e809-1306-4cd1-8ac1-ebfdf7c872b8,I sit with a boy in english he became my frien...,F,18-24
...,...,...,...,...
147,f3af2ca2-dbfb-4447-a4f2-dfdfdda58640,@username what about Robin Ruiter?\t\t Thumbin...,M,18-24
148,f707b0c7-f11f-480c-b859-c38ecfa201e2,Glamour Moddel ???? had to be one lol\t\t Did ...,M,18-24
149,f7ea5331-febf-4ed4-9416-7353c898ad35,Just posted a photo http://t.co/96nT74JE\t\t I...,M,25-34
150,fd7c89ad-deb5-4445-99c5-bf4c698ee371,@username sactan ne haber?\t\t Aurora borealis...,F,18-24


## Creating feature functions

In [9]:
import re
from nltk.probability import FreqDist
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import statistics


# Character-based features
def character_count(text):
    return len(text)

def alphabetic_ratio(text):
    alphabetic = sum(c.isalpha() for c in text)
    return alphabetic/len(text)

def uppercase_ratio(text):
    upper = sum(c.isupper() for c in text)
    return upper/len(text)

def digit_ratio(text):
    digit = sum(c.isdigit() for c in text)
    return digit/len(text)

def whitespace_ratio(text):
    whitespace = sum(c.isspace() for c in text)
    return whitespace/len(text)

def tab_ratio(text):
    tabs = text.count('\t')
    return tabs/len(text)

def letter_ratio(text, letter):
    text = text.lower()
    letter_count = text.count(letter)
    return letter_count/len(text)

def specialcharacter_ratio(text, character):
    spec_count = text.count(character)
    return spec_count/len(text)

# Word-based features
def number_words(text):
    words = re.findall(r'\b\w+\b', text)
    return len(words)

def word_length(text):
    words = re.findall(r'\b\w+\b', text)
    num_words = len(words)
    if num_words == 0:
        return 0

    total_length = sum(len(word) for word in words)
    return total_length/num_words

def vocabulary_richness(text):
    words = re.findall(r'\b\w+\b', text)
    num_words = len(words)
    if num_words == 0:
        return 0
    
    num_uniq_words = len(set(words))
    return num_uniq_words/num_words

def long_words(text):
    words = re.findall(r'\b\w+\b', text)
    long_words_list = [word for word in words if len(word) > 6]
    return len(long_words_list)/len(words)

def short_words(text):
    words = re.findall(r'\b\w+\b', text)
    short_words_list = [word for word in words if 1 <= len(word) <= 3]
    return len(short_words_list)/len(words)

def legomena(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    legomena = [word for word in freq if freq[word] == 1]
    return len(legomena)/len(words)

def dislegomena(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    dislegomena = [word for word in freq if freq[word] == 2]
    return len(dislegomena)/len(words)

def yules_k(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    N = len(words)
    Vi = FreqDist(freq.values())
    K = 10**4 * ((-N + sum(i**2 * Vi[i] for i in Vi))/N**2)
    return K

def simpson_d(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    N = len(words)
    if N < 2:
        return 0
    D = sum(fr * (fr - 1) / (N * (N - 1)) for fr in freq.values())
    return D

def sichel_s(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    dislegomena = [word for word in freq if freq[word] == 2]
    S = len(dislegomena)/len(freq.values())
    return S

def honores_r(text):
    words = re.findall(r'\b\w+\b', text.lower())
    if not words:
        return 0
    freq = FreqDist(words)
    N = len(words)
    V = len(freq.values())
    legomena = [word for word in freq if freq[word] == 1]
    unique_count_ratio = len(legomena) / V if V > 0 else 0
    if unique_count_ratio == 1 or N == 0:
        return 0
    R = (100*np.log(N)/(1-(len(legomena)/V)))
    return R

def entropy(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    N = len(words)
    E = -sum((fr / N) * np.log(fr/N) for fr in freq.values())
    return E

# Syntatic features
def punctuations_ratio(text, punctuation):
    punctuation_list = re.findall(punctuation, text)
    return len(punctuation_list)/len(text)

# Structural features
def lines(text):
    return len(text.split('\n'))

def sentences(text):
    return len(sent_tokenize(text))

def paragraphs(text):
    return len([par for par in text.split('\n\n') if par.strip()])

def sentence_paragraph(text):
    pars = [par for par in text.split('\n\n') if par.strip()]
    return statistics.mean([len(sent_tokenize(par)) for par in pars])

def words_paragraph(text):
    pars = [par for par in text.split('\n\n') if par.strip()]
    return statistics.mean([len(re.findall(r'\b\w+\b', par)) for par in pars])

def chars_paragraph(text):
    pars = [par for par in text.split('\n\n') if par.strip()]
    return statistics.mean([len(par) for par in pars])

def words_sentences(text):
    sents = sent_tokenize(text)
    return statistics.mean([len(word_tokenize(sentence)) for sentence in sents])

def uppercase_start(text):
    sents = sent_tokenize(text)
    return (sum(1 for sentence in sents if sentence[0].isupper()) / len(sents))


In [10]:
def extract_features(dataframe, text_column):
    features = pd.DataFrame()

    # Character-based features
    features['total_characters'] = dataframe[text_column].apply(character_count)
    features['ratio_alphabetic'] = dataframe[text_column].apply(alphabetic_ratio)
    features['ratio_uppercase'] = dataframe[text_column].apply(uppercase_ratio)
    features['ratio_digit'] = dataframe[text_column].apply(digit_ratio)
    features['ratio_whitespace'] = dataframe[text_column].apply(whitespace_ratio)
    features['ratio_tabspace'] = dataframe[text_column].apply(tab_ratio)
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    for letter in letters:
        features[letter+'_frequency'] = dataframe[text_column].apply(letter_ratio, args=(letter,))
    special_characters = ['~', '@', '#', '$', '%', '^', '&', '*', '-', '_', '=', '+', '>', '<', '[', ']', '{', '}', '/', '\\', '|']
    for character in special_characters:
        features[character+'_frequency'] = dataframe[text_column].apply(specialcharacter_ratio, args=(character,))
    
    # Word-based features
    features['total_words'] = dataframe[text_column].apply(number_words)
    features['word_length'] = dataframe[text_column].apply(word_length)
    features['vocabulary_richness'] = dataframe[text_column].apply(vocabulary_richness)
    features['long_words'] = dataframe[text_column].apply(long_words)
    features['short_words'] = dataframe[text_column].apply(short_words)
    features['hapax_legomena'] = dataframe[text_column].apply(legomena)
    features['hapax_dislegomena'] = dataframe[text_column].apply(dislegomena)
    features['yules_k'] = dataframe[text_column].apply(yules_k)
    features['simpson_d'] = dataframe[text_column].apply(simpson_d)
    features['sichel_s'] = dataframe[text_column].apply(sichel_s)
    #features['honore_r'] = dataframe[text_column].apply(honores_r)
    features['entropy'] =  dataframe[text_column].apply(entropy)
    # Brunet W?
    # word length frequency distribution

    
    # Syntactic features
    punctuations = [r"’", r",", r"\.", r":", r";", r"\?", r"\?{2,}", r"!", r"!{2,}", r"\.{3}"]
    for punctuation in punctuations:
        features[punctuation+"_frequency"] = dataframe[text_column].apply(punctuations_ratio, args=(punctuation,))

    # Structural features
    features['number_lines'] = dataframe[text_column].apply(lines)
    features['number_sentences'] = dataframe[text_column].apply(sentences)
    features['number_paragraphs'] = dataframe[text_column].apply(paragraphs)
    features['sentences_per_paragraph'] = dataframe[text_column].apply(sentence_paragraph)
    features['word_per_paragraph'] = dataframe[text_column].apply(words_paragraph)
    features['character_per_paragraph'] = dataframe[text_column].apply(chars_paragraph)
    features['word_per_sentence'] = dataframe[text_column].apply(words_sentences)
    features['ratio_sentencestart_uppercase'] = dataframe[text_column].apply(uppercase_start)
    #features['gender'] = dataframe['gender']
        
    return features

In [48]:
pan15_train = extract_features(data_pan15_train, 'text')

In [49]:
pan15_test = extract_features(data_pan15_test, 'text')

In [113]:
pan17_train = extract_features(data_pan17_train, 'text')

In [115]:
pan17_test = extract_features(data_pan17_test, 'text')

In [11]:
combined_pan15_train_features = extract_features(combined_pan15_train, 'text')
combined_pan15_test_features = extract_features(combined_pan15_test, 'text')

In [9]:
combined_pan17_train_features = extract_features(combined_pan17_train, 'text')

In [10]:
combined_pan17_test_features = extract_features(combined_pan17_test, 'text')

In [12]:
combined_pan15_test_features

,total_characters,ratio_alphabetic,ratio_uppercase,ratio_digit,ratio_whitespace,ratio_tabspace,a_frequency,b_frequency,c_frequency,d_frequency,...,"!{2,}_frequency",\.{3}_frequency,number_lines,number_sentences,number_paragraphs,sentences_per_paragraph,word_per_paragraph,character_per_paragraph,word_per_sentence,ratio_sentencestart_uppercase
0,12104,0.768341,0.061715,0.013219,0.172753,0.016523,0.066590,0.011484,0.027098,0.027677,...,0.000165,0.000000,13,120,6,20.666667,328.500000,2015.666667,18.758333,0.700000
1,6745,0.719644,0.045960,0.006672,0.203558,0.029652,0.053225,0.011119,0.017198,0.024314,...,0.000000,0.004596,2,96,1,96.000000,1219.000000,6745.000000,15.885417,0.635417
2,6561,0.660113,0.057918,0.003048,0.208352,0.030483,0.055327,0.010364,0.015394,0.020881,...,0.000000,0.000000,6,226,1,226.000000,1027.000000,6561.000000,8.088496,0.261062
3,6924,0.725881,0.161612,0.021375,0.172733,0.028885,0.062103,0.012276,0.027585,0.023252,...,0.000000,0.000578,15,45,4,11.750000,291.000000,1729.500000,29.155556,0.244444
4,1639,0.731544,0.108603,0.001830,0.214765,0.031727,0.055522,0.007322,0.019524,0.021354,...,0.000610,0.000610,2,25,1,25.000000,312.000000,1639.000000,15.000000,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,4199,0.722553,0.092165,0.012146,0.189807,0.038104,0.071684,0.010479,0.021910,0.020005,...,0.000238,0.005477,1,68,1,68.000000,681.000000,4199.000000,12.573529,0.617647
138,6482,0.690528,0.072046,0.015273,0.169701,0.024375,0.053379,0.013422,0.020056,0.021907,...,0.001388,0.001543,11,137,1,137.000000,1063.000000,6482.000000,11.335766,0.335766
139,2749,0.719534,0.102219,0.021462,0.153147,0.019644,0.061477,0.010186,0.020007,0.018188,...,0.000364,0.001091,34,40,14,3.214286,29.428571,194.500000,13.875000,0.325000
140,5281,0.697216,0.100360,0.048854,0.160765,0.017042,0.036167,0.009468,0.016285,0.013255,...,0.000000,0.000000,1,50,1,50.000000,933.000000,5281.000000,21.980000,0.420000


In [50]:
pan15_test.to_csv(r'data\pan15_features_test.csv', index=False)

In [51]:
pan15_train.to_csv(r'data\pan15_features_training.csv', index=False)

In [119]:
pan17_test.to_csv(r'data\pan17_features_test.csv', index=False)

In [120]:
pan17_train.to_csv(r'data\pan17_features_training.csv', index=False)

## Create TF-IDF

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [121]:
tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

In [94]:
pan15_train_tfidf_array = tfidf.fit_transform(data_pan15_training['text'])
pan15_test_tfidf_array = tfidf.transform(data_pan15_test['text'])

In [123]:
pan17_train_tfidf_array = tfidf.fit_transform(data_pan17_train['text'])
pan17_test_tfidf_array = tfidf.transform(data_pan17_test['text'])

In [124]:
svd = TruncatedSVD(n_components=100, random_state=42)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

In [96]:
pan15_train_lsa = lsa.fit_transform(pan15_train_tfidf_array)
pan15_test_lsa = lsa.transform(pan15_test_tfidf_array)

In [125]:
pan17_train_lsa = lsa.fit_transform(pan17_train_tfidf_array)
pan17_test_lsa = lsa.transform(pan17_test_tfidf_array)

In [78]:
feature_names = tfidf.get_feature_names_out()
pan15_train_tfidf = pd.DataFrame(pan15_train_tfidf_array.toarray(), columns=feature_names)
pan15_test_tfidf = pd.DataFrame(pan15_test_tfidf_array.toarray(), columns=feature_names)

In [79]:
pan15_train_tfidf.to_csv(r'data\pan15_tfidf_train.csv', index=False)
pan15_test_tfidf.to_csv(r'data\pan15_tfidf_test.csv', index=False)

## Classifier

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [98]:
len(data_pan15_training)

14166

In [99]:
X_train = pan15_train_lsa
y_train = data_pan15_training['gender']
X_test = pan15_test_lsa
y_test = data_pan15_test['gender']

In [126]:
X_train = pan17_train_lsa
y_train = data_pan17_train['gender']
X_test = pan17_test_lsa
y_test = data_pan17_test['gender']

In [19]:
X_train = combined_pan17_train_features
y_train = combined_pan17_train['gender']
X_test = combined_pan17_test_features
y_test = combined_pan17_test['gender']

In [25]:
normalizer = Normalizer(copy=False)
model = LinearSVC()
norm_model = make_pipeline(normalizer, model)

In [26]:
norm_model.fit(X_train, y_train)

Pipeline(steps=[('normalizer', Normalizer(copy=False)),
                ('linearsvc', LinearSVC())])

In [27]:
pred = model.predict(X_test)
accuracy_score(y_test, pred)

c:\Users\Sten\AppData\Local\R-MINI~1\envs\python39\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but LinearSVC was fitted without feature names
  warnings.warn(


0.5470833333333334

## Classifier 2

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [50]:
class CustomFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, text_column):
        self.text_column = text_column
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return extract_features(X, self.text_column)

In [16]:
char_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), sublinear_tf=True, min_df=2)
#char_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), min_df=2)
word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), sublinear_tf=True, min_df=2)
#word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2)
#stylometric_features = CustomFeatureExtractor(text_column='text')

combined_features = FeatureUnion([
    #('stylometric', stylometric_features),
    ('char', char_vectorizer),
    ('word', word_vectorizer)
])

pipeline = Pipeline([
    ('features', combined_features),
    ('svm', LinearSVC())
])

In [78]:
X_train = combined_pan15_train['text']
y_train = combined_pan15_train['gender']
X_test = combined_pan15_test['text']
y_test = combined_pan15_test['gender']

In [79]:
output = combined_features.fit_transform(X_train)

In [80]:
output_test = combined_features.transform(X_test)

In [66]:
X_train = output
X_test = output_test

In [122]:
scaler = StandardScaler()
combined_pan17_train_features_scaled = pd.DataFrame(scaler.fit_transform(combined_pan17_train_features), columns=combined_pan17_train_features.columns)
combined_pan17_test_features_scaled = pd.DataFrame(scaler.transform(combined_pan17_test_features), columns=combined_pan17_test_features.columns)

In [81]:
scaler = StandardScaler()
combined_pan15_train_features_scaled = pd.DataFrame(scaler.fit_transform(combined_pan15_train_features), columns=combined_pan15_train_features.columns)
combined_pan15_test_features_scaled = pd.DataFrame(scaler.transform(combined_pan15_test_features), columns=combined_pan15_test_features.columns)

In [89]:
X_train = hstack([output, csr_matrix(combined_pan15_train_features_scaled)])
X_test = hstack([output_test, csr_matrix(combined_pan15_test_features_scaled)])

In [74]:
X_train = combined_pan17_train_features_scaled
X_test = combined_pan17_test_features_scaled

NameError: name 'combined_pan17_train_features_scaled' is not defined

In [75]:
X_train = combined_pan15_train_features_scaled
X_test = combined_pan15_test_features_scaled

In [98]:
model = LinearSVC(max_iter=10000, C= 0.01, class_weight= None, loss= 'squared_hinge', tol= 0.0001)
model.fit(X_train, y_train)

LinearSVC(C=0.01, max_iter=10000)

In [100]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7535211267605634

In [88]:
param_grid= {
    'C': [0.01, 0.1, 1, 10, 100],
    'tol': [1e-4, 1e-5],
    'loss': ['hinge', 'squared_hinge'],
    'class_weight': [None, 'balanced']
}
model = LinearSVC(max_iter=10000)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

In [90]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5, estimator=LinearSVC(max_iter=10000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'class_weight': [None, 'balanced'],
                         'loss': ['hinge', 'squared_hinge'],
                         'tol': [0.0001, 1e-05]},
             scoring='accuracy', verbose=2)

In [93]:
grid_search.best_params_

{'C': 0.01, 'class_weight': None, 'loss': 'squared_hinge', 'tol': 0.0001}

In [67]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('char',
                                                 TfidfVectorizer(analyzer='char',
                                                                 min_df=2,
                                                                 ngram_range=(3,
                                                                              5),
                                                                 sublinear_tf=True)),
                                                ('word',
                                                 TfidfVectorizer(min_df=2,
                                                                 ngram_range=(1,
                                                                              2),
                                                                 sublinear_tf=True))])),
                ('svm', LinearSVC())])

In [68]:
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.8195833333333333

In [2]:
combined_pan17_train = data_pan17_train.groupby('author').agg({
    'text': ' '.join,
    'gender': 'first'
}).reset_index()
combined_pan17_test = data_pan17_test.groupby('author').agg({
    'text': ' '.join,
    'gender': 'first'
}).reset_index()

NameError: name 'data_pan17_train' is not defined

In [104]:
combined_pan15_train.loc[0, 'text']

'Things I want for my business cards but are too expensive: 3 PMS colors. colored edges, soft touch finish, raised spot UV, cut 45°corners.\t\t "painters produced their most highly valued work when they were 41.92 years old"\nhttp://t.co/T8Gvom70GE\n\nSo... I\'ve 15 years left for mine?\t\t @username your new discussion layout is confusing regarding who says what, because the comments aren\'t sectioned off. http://t.co/Dfkg4js2o2\t\t I never really understood why game environments (which you see most of) get so few resources compared to characters. https://t.co/3Af3fn7XVe\t\t @username 20k and 2048² on a gun, fine. But throwing the same on an enemy that you try to shoot before he gets too close?\t\t @username Guns using the texture already in memory is good. Guns on the floor having the same model as FPS isn\'t. Probably LOD models.\t\t @username I\'m talking about chars that are small scale compared to enviro props. Extreme example: http://t.co/HOR8W8T2Pr 30~40tri scene\t\t @username 